## 线性回归 (linear regression)
我们有一组输入变量(x)用于确定输出变量(y).输入变量和输出变量之间存在关系。 通过线性回归的目标是量化这种关系。
在线性回归中，输入变量（x）和输出变量（y）之间的关系表示为y = a + bx形式的等式。因此，线性回归的目标是找出系数a和b的值



### 一元线性回归
比如下面的一个数据集分布图(picture1), 通过蓝色点的分布情况， 可以大致看成是条线段。  
希望通过线性回归找到a, b的关系，得到成y=ax+b关系的数据集(x, y)绘制成一条红线，使得蓝色的点尽可能均匀的分布在红线的上下方:
![picture1](images/1_1.png)

In [2]:
import pandas as pd
import random
import plotly.offline as offline
import plotly.graph_objs as go  

offline.init_notebook_mode(connected=True)

def generate_data(n):
    dataset = []
    """y = 2x + random(5-10)"""
    for i in range(1, n):
        y = 2 * i + random.randint(10, 20)
        dataset.append([i, y])
    return dataset

def generate_forecast_data(n, k, b):
    """n: size
       k: 斜率
       b: 常数项
    """
    dataset = []
    for i in range(1, n):
        y = k * i + b
        dataset.append([i, y])
    return dataset

# 生成随机数据
df = pd.DataFrame(generate_data(20), columns=['x', 'y'], dtype='float64')

# 生成假设的数据
df1 = pd.DataFrame(generate_forecast_data(20, 2, 10), columns=['x', 'y'], dtype='float64')


dataset = go.Scatter(
    x = df['x'],
    y = df['y'],
    name = 'dataset',
    mode = 'markers'
)
predict = go.Scatter(
    x = df1['x'],
    y = df1['y'],
    name = 'predict',
    mode = 'lines'
)

figs = [dataset, predict]
offline.iplot({'data': figs, 'layout': {'title': 'picture1', 'font': dict(size=16)}})

#### 处理思路
针对方程的未知参数，一般采用最小二乘法进行估计。    
***最小二乘法：利用最小二乘法可以简便地求得未知的数据，并使得这些求得的数据与实际数据之间误差的平方和为最小。***    
   
回到上面讨论的问题， 数据集线性公式为y=ax + b 需要求出a, b， 通过最小二乘法求a, b的公式为:  
$\Large a=\frac{\sum_{i=1}^{n}x_{i}y_{i} - n \cdot \overline{x}\cdot\overline{y}}{\sum_{i=i}^{n} \cdot x_{i}^{2} - n\cdot(\overline{x})^{2}}$  
$\Large b = \overline{y} - a\overline{x}$


#### 代码实现

In [4]:
def leastsq(x, y):
    """x为x值的集合
       y为y值的集合
    """
    meanx = sum(x) / len(x)   #求x的平均值
    meany = sum(y) / len(y)   #求y的平均值

    xsum = 0.0
    ysum = 0.0

    for i in range(len(x)):
        xsum += (x[i] - meanx)*(y[i]-meany)
        ysum += (x[i] - meanx)**2

    k = xsum/ysum
    b = meany - k*meanx

    return k,b   

k, b = leastsq(df['x'], df['y'])
df2 = pd.DataFrame(generate_forecast_data(20, round(k, 2), round(b, 2)), columns=['x', 'y'], dtype='float64')

cal_dataset = go.Scatter(
        x= df2['x'],
        y= df2['y'],
        mode = 'lines',
        name = 'calculate_dataset'
    )

data2 = [dataset, cal_dataset]
offline.iplot({"data": data2,  'layout': {'title': 'picture2',
                          'font': dict(size=16)}})

从图picture2可以看出，通过最小二乘法求得得斜率和常数项， 使得蓝色的点均分分布在它生成得红线上下方。
![picture1](images/1_2.png)

##### 计算决定系数
y = ax + b   
决定系数$r^{2}$： 自变量(x)能够描述多少百分比的因变量(y), 公式:  
$R^{2}=1 - \frac{SS_{res}}{SS_{tot}}$  
$SS_{res} = \sum_{i}(y_{i} - f_{i})^{2}$   残差的平方和       
$SS_{tot} = \sum_{i}(y_{i} - \overline{y})^{2}$   平方和(与数据的方差成比例)    
   
*** 也就是说残差越小，决定系数$R^{2}$ 越接近1， 拟合的直线更能解释因变量Y的波动***


In [22]:
from math import pow
def calculate_determination(dataset, cal_dataset):
    y = df.mean()['y']   # y的平均值
    ss_res = 0
    ss_tot = 0
    for i in range(len(dataset)):
        ss_res += pow((dataset.iloc[i]['y'] - cal_dataset.iloc[i]['y']), 2)
        ss_tot += pow((dataset.iloc[i]['y'] - y), 2)
    return 1 - (ss_res / ss_tot)

calculate_determination(df, df2)

0.930457162836733